In [239]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import scipy.stats as st


def ess(estimate_v, sample_v):
    sample_mean = np.mean(sample_v)
    return sum((estimate_v[i] - sample_mean)**2 for i in range(len(estimate_v)))


def rss(estimate_v, sample_v):
    return sum((estimate_v[i] - sample_v[i])**2 for i in range(len(estimate_v)))


def tss(sample_v):
    sample_mean = np.mean(sample_v)
    return sum((sample_v[i] - sample_mean)**2 for i in range(len(sample_v)))

In [197]:
data = pd.read_csv("data.csv", usecols=[
                   'class1_v10', 'class2_v10', 'class3_v10', 'sex_v10', 'survived_v10'])
data.rename(inplace=True, columns={'class1_v10': 'class1',
                                   'class2_v10': 'class2',
                                   'class3_v10': 'class3',
                                   'sex_v10': 'sex',
                                   'survived_v10': 'survived'})
data = data[pd.notnull(data['survived'])]
data['intercept'] = 1

Имеется выборка из взрослых пассажиров титаника, у каждого следующие характеристики:  
survived - выжил ли пассажир (1, если да, иначе 0)  
sex - пол (1 для мужчин, 0 для женщин)  
class1 - 1 для пассажиров 1 класса, иначе 0  
class2 - 1 для пассажиров 2 класса, иначе 0  
class3 - 1 для пассажиров 3 класса, иначе 0  

# Часть I
### Оценим модель логит-регрессии для вероятности выжить в зависимости от пола и класса:  
$\frac{P(survived=1)}{P(survived=0)}=exp(\beta_1 + \beta2\cdot sex + \beta_3\cdot class1+\beta_4\cdot class2)$

In [330]:
# Третий регрессор class3 не рассматривается, т.к. между
# class1, class2 и class3 существует мультиколлинеарность.
logit_model = smf.logit(formula="survived ~ sex + class1 + class2", data=data)
logit_result = logit_model.fit()
logit_fitted_list = [1 if f > 0 else 0 for f in logit_result.fittedvalues]
logit_result.summary()

Optimization terminated successfully.
         Current function value: 0.468990
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                 1131
Model:                          Logit   Df Residuals:                     1127
Method:                           MLE   Df Model:                            3
Date:                Tue, 06 Jun 2017   Pseudo R-squ.:                  0.2849
Time:                        13:25:45   Log-Likelihood:                -530.43
converged:                       True   LL-Null:                       -741.76
                                        LLR p-value:                 2.737e-91
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3926      0.138      2.855      0.004       0.123       0.662
sex           -2.5395      0.159    -15.978      0.000      -2.851      -2.228
class1         1.6668      0.182      9.134      0.000       1.309       2.024
class2         0.5534      0.196      2.824      0.005       0.169       0.937
==============================================================================
"""

Проверим значимость модели регресси в целом.  
$H_0: \beta_2=\beta_3=\beta_4=0$  
Для этого ипользуем тест отношения правдоподобия:  
$LR = -2\log(\frac{L\:|\: H_0}{L\:|\:H_A}) \sim \chi^2_k$, где $L$ — значение функции правдоподобия модели  
$L = \prod_{i=1}^{n}p^{Y_i}(1-p)^{1-Y_i} = p^{\sum_{i=1}^nY_i}(1-p)^{n-\sum_{i=1}^nY_i}$

In [307]:
n = len(data['survived']) # количество элементов в выборке
k = 4 # количество оцениваемых коэффициентов (включая свободный)
LR = -2*(logit_result.llnull - logit_result.llf)
LR_crit = st.chi2.ppf(0.95, k)
print("LR =", LR)
print("LR_crit =", LR_crit)

LR = 422.65656195
LR_crit = 9.48772903678


$LR > LR_{crit}$, значит, отвергаем нулевую гипотезу.
$\:$  
$\:$  
$\:$  
### Далее оценим модель пробит-регрессии:  
$P(survived=1) = Ф(\beta_1 + \beta_2\cdot sex + \beta_3\cdot class1+\beta_4\cdot class2)$,  
где Ф - функция нормального распределения  
$Ф^{-1}(P(survived=1)) = \beta_1 + \beta_2\cdot sex + \beta_3\cdot class1+\beta_4\cdot class2$

In [329]:
probit_model = smf.probit(formula="survived ~ sex + class1 + class2", data=data)
probit_result = probit_model.fit()
probit_fitted_list = [1 if f > 0 else 0 for f in probit_result.fittedvalues]
probit_result.summary()

Optimization terminated successfully.
         Current function value: 0.469734
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                 1131
Model:                         Probit   Df Residuals:                     1127
Method:                           MLE   Df Model:                            3
Date:                Tue, 06 Jun 2017   Pseudo R-squ.:                  0.2838
Time:                        13:25:39   Log-Likelihood:                -531.27
converged:                       True   LL-Null:                       -741.76
                                        LLR p-value:                 6.338e-91
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2835      0.082      3.439      0.001       0.122       0.445
sex           -1.5190      0.091    -16.689      0.000      -1.697      -1.341
class1         0.9567      0.104      9.167      0.000       0.752       1.161
class2         0.2738      0.113      2.418      0.016       0.052       0.496
==============================================================================
"""

### Оценим линейную модель:
$P(survived=1) = \beta_1 + \beta_2\cdot sex + \beta_3\cdot class1+\beta_4\cdot class2$

In [336]:
lin_model = smf.ols(formula="survived ~ sex + class1 + class2", data=data)
lin_result = lin_model.fit()
lin_fitted_list = [1 if f > 0.5 else 0 for f in lin_result.fittedvalues]
lin_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               survived   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     201.8
Date:                Tue, 06 Jun 2017   Prob (F-statistic):          9.59e-105
Time:                        13:51:07   Log-Likelihood:                -534.45
No. Observations:                1131   AIC:                             1077.
Df Residuals:                    1127   BIC:                             1097.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6091      0.024     25.447      0.000       0.562       0.656
sex           -0.5091      0.025    -20.602      0.000      -0.558      -0.461
class1         0.2802      0.028      9.994      0.000       0.225       0.335
class2         0.0844      0.030      2.822      0.005       0.026       0.143
==============================================================================
Omnibus:                       64.857   Durbin-Watson:                   1.679
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               74.910
Skew:                           0.626   Prob(JB):                     5.41e-17
Kurtosis:                       3.151   Cond. No.                         4.24
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Сравним модели логит, пробит и линейную:
Проверяем, все ли прогнозы трёх моделей одинаковые, и выводим первые 30 результатов

In [355]:

survived_list = list(data['survived'])
sex_list = list(data['sex'])
class1_list = list(data['class1'])
class2_list = list(data['class2'])

different = False
for i in range(100):
    if(0 < logit_fitted_list[i] + probit_fitted_list[i] + lin_fitted_list[i] < 3):
        different = True

print("Models offer different results:",different)
print("Fitted values:")
print("Sex\t\tClass1\t\tClass2\t\tSurvived\t\tLogit\t\tProbit\t\tLinear")
for i in range(30):
    print(sex_list[i], class1_list[i], class2_list[i],
          survived_list[i], logit_fitted_list[i], probit_fitted_list[i], lin_fitted_list[i], sep="\t\t")

Models offer different results: False
Fitted values:
Sex		Class1		Class2		Survived		Logit		Probit		Linear
1.0		0.0		0.0		0.0		0		0		0
1.0		1.0		0.0		1.0		0		0		0
1.0		1.0		0.0		1.0		0		0		0
1.0		1.0		0.0		1.0		0		0		0
1.0		0.0		0.0		1.0		0		0		0
1.0		0.0		0.0		0.0		0		0		0
1.0		1.0		0.0		0.0		0		0		0
0.0		1.0		0.0		1.0		1		1		1
0.0		1.0		0.0		1.0		1		1		1
1.0		0.0		0.0		0.0		0		0		0
1.0		1.0		0.0		0.0		0		0		0
1.0		0.0		0.0		1.0		0		0		0
1.0		0.0		0.0		0.0		0		0		0
0.0		1.0		0.0		1.0		1		1		1
1.0		0.0		0.0		0.0		0		0		0
1.0		1.0		0.0		0.0		0		0		0
1.0		0.0		0.0		0.0		0		0		0
1.0		0.0		0.0		0.0		0		0		0
1.0		0.0		1.0		0.0		0		0		0
1.0		0.0		1.0		0.0		0		0		0
0.0		1.0		0.0		1.0		1		1		1
0.0		0.0		1.0		1.0		1		1		1
0.0		1.0		0.0		1.0		1		1		1
1.0		1.0		0.0		0.0		0		0		0
1.0		0.0		0.0		0.0		0		0		0
0.0		1.0		0.0		1.0		1		1		1
1.0		0.0		1.0		0.0		0		0		0
1.0		0.0		0.0		0.0		0		0		0
0.0		0.0		1.0		1.0		1		1		1
1.0		1.0		0.0		0.0		0		0		0


Итак, расхождений между прогнозами нет вообще. Проверим, сколько прогнозов неверны:

In [353]:
print(str(int(rss(logit_fitted_list,survived_list))) + " / " + str(n))

238 / 1131


Итак, из 1131 элемента выборки результаты 238 предсказаны неверно. Хороший результат.

# Часть II
### Оценим расширенную логит-регрессию, включающую вклад пола в шансы выживания в зависимости от класса:
$\frac{P(survived=1)}{P(survived=0)}=exp(\beta_1 + \beta2\cdot sex + \beta_3\cdot class1+\beta_4\cdot class2 + 
\beta_5 \cdot sex \cdot class1 + \beta_6 \cdot sex \cdot class2)$

In [366]:
logit_ext_model = smf.logit(formula="survived ~ sex + class1 + class2 + sex * class1 + sex * class2", data=data)
logit_ext_result = logit_ext_model.fit()
logit_ext_fitted_list = [1 if f > 0 else 0 for f in logit_ext_result.fittedvalues]
logit_ext_result.summary()

Optimization terminated successfully.
         Current function value: 0.444751
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                 1131
Model:                          Logit   Df Residuals:                     1125
Method:                           MLE   Df Model:                            5
Date:                Tue, 06 Jun 2017   Pseudo R-squ.:                  0.3219
Time:                        14:35:08   Log-Likelihood:                -503.01
converged:                       True   LL-Null:                       -741.76
                                        LLR p-value:                5.776e-101
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1579      0.156     -1.011      0.312      -0.464       0.148
sex           -1.5280      0.205     -7.471      0.000      -1.929      -1.127
class1         3.6391      0.531      6.852      0.000       2.598       4.680
class2         1.8970      0.339      5.598      0.000       1.233       2.561
sex:class1    -2.7417      0.573     -4.783      0.000      -3.865      -1.618
sex:class2    -2.4848      0.459     -5.409      0.000      -3.385      -1.584
==============================================================================
"""

In [359]:
rss(logit_ext_fitted_list, survived_list)

225.0

Такая модель предсказывает чуть лучше: 225 неверных прогнозов из 1131.

### Проверим гипотезу $H_0:\beta_5 = \beta_6 = 0$
т.е. $Q\cdot\beta = r$, где:  
$Q = \begin{pmatrix}
 0&0&0&0&0&1\\ 
 0&0&0&0&1&-1
\end{pmatrix} \qquad
r = \begin{pmatrix}
0\\
0
\end{pmatrix}$

In [363]:
Q = np.array([[0,0,0,0,0,1],[0,0,0,0,1,-1]])
r = np.array([[0],[0]])

### с помощью теста Вальда:
$W = (Q \cdot \widehat{\beta} - r)^T \cdot (Q \cdot \widehat{V}(\widehat{\beta}) \cdot Q^T)^{-1} \cdot (Q \cdot \widehat{\beta} - r) \sim \chi^2_q$, где:  
$q$ — количество ограничений (количество строк матрицы Q)  
$\widehat{V}(\widehat{\beta})$ — оценка ковариационной матрицы коэффициентов модели регрессии

In [380]:
#Q = np.array([[0,0,0,0,0,1],[0,0,0,0,1,-1]])
#r = np.array([[0],[0]])
#logit_ext_result.wald_test(r_matrix=(Q,r)
hyp = '(sex:class1 = 0), (sex:class2 = 0)'
W = logit_ext_result.wald_test(hyp).statistic[0][0]
W_crit = st.chi2.ppf(0.95, 2)
print("W =",W)
print("W_crit =", W_crit)

W = 45.0498955004
W_crit = 5.99146454711


$W > W_{crit}$, значит, отвергаем нулевую гипотезу.

### c помощью отношения правдоподобия:
$LR = -2\log(\frac{L\:|\: H_0}{L\:|\:H_A}) \sim \chi^2_k$, где $L$ — значение функции правдоподобия модели  

In [386]:
LR_2 = -2 * (logit_result.llf - logit_ext_result.llf)
LR_2_crit = st.chi2.ppf(0.95, k)
print("LR =",LR_2)
print("LR_crit =", LR_crit)


LR = 54.8275653611
LR_crit = 9.48772903678


$LR > LR_{crit}$, значит, отвергаем нулевую гипотезу.  
Т.е. можно предполагать, что, действительно, роль пола связана с классом каюты.  
###Чтобы сравнить модели, переберём все возможные сочетания класса и пола:

In [407]:
sex = [0, 0, 0, 1, 1, 1]
class1 = [1, 0, 0, 1, 0, 0]
class2 = [0, 1, 0, 0, 1, 0]
df = pd.DataFrame({'sex': sex, 'class1': class1, 'class2': class2})
logit_pred = logit_result.predict(df)
#logit_pred = [1 if f > 0.5 else 0 for f in logit_pred]
logit_ext_pred = logit_ext_result.predict(df)
#logit_ext_pred = [1 if f > 0.5 else 0 for f in logit_ext_pred]

print("Predictions:")
print("Sex\t\tClass1\t\tClass2\t\tLogit pred.\tExt. logit pred.")
for i in range(len(sex)):
    print(sex[i], class1[i], class2[i], logit_pred[i],
          logit_ext_pred[i], sep="\t\t")

Predictions:
Sex		Class1		Class2		Logit pred.	Ext. logit pred.
0		1		0		0.886893028104		0.970149253731
0		0		1		0.720305567695		0.850574712644
0		0		0		0.596907574816		0.460606060606
1		1		0		0.382227088963		0.3125
1		0		1		0.168889437404		0.0933333333333
1		0		0		0.104621264725		0.15632183908


Итак, мы наблюдаем, что основная спецификация показывает мало шансов на спасение у мужчин, но высокие шансы на спасение у женщин. Расширенная же спецификация говорит о том, что у женщин 3 класса тоже немного шансов выжить (около 46% женщин третьего класса спасаются).